In [1]:
from ilqr import ILQR
import gym
import sys
import env
import jax.numpy as np
from tqdm.notebook import trange

env = gym.make('CartPoleContinuous-v0').env

/home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'env.cartpole_continuous.CartPoleContinuousEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
obs = env.reset()


def final_cost(x):
    return 0.5 * (np.square(1.0 - np.cos(x[2])) + np.square(x[1]) + np.square(x[3]))


def running_cost(x, u):
    return 0.5 * np.sum(np.square(u))


def model(x, u):
    return env._state_eq(x, u)


horizon = 100
per_iter = 5
u_range = [-env.max_force, env.max_force]

ilqr = ILQR(final_cost, running_cost, model, u_range, horizon, per_iter)

u_seq = [np.zeros(1) for _ in range(horizon)]
x_seq = [obs.copy()]
for t in range(ilqr.horizon):
    x_seq.append(env._state_eq(x_seq[-1], u_seq[t]))


ctrl_seq = []

for i in trange(500):
#     env.render(mode='Human')
    u_seq = ilqr.predict(x_seq, u_seq)

    ctrl_seq.append(u_seq[0])
    obs, _, _, _ = env.step(u_seq[0])
    x_seq[0] = obs.copy()


KeyboardInterrupt: 

In [3]:
env.reset()

for ctrl in ctrl_seq:
    env.render(mode='Human')
    env.step(ctrl)